**ФИО:**

# Задание 1

**Описание:** Создайте иерархию классов для разных типов сотрудников в компании. Реализуйте родительский класс Employee и дочерние классы Manager и Developer. Каждый класс должен иметь метод для расчета зарплаты на основе различных критериев класса.


Отрабатываемый принцип: Наследование



In [ ]:
class Employee:
    vacation_days = 28

    def __init__(self, first_name, second_name, gender):
        self.first_name = first_name
        self.second_name = second_name
        self.gender = gender
        self.remaining_vacation_days = Employee.vacation_days
        self._employee_id = self.__generate_employee_id()

    def consume_vacation(self, days):
        self.remaining_vacation_days -= days

    def get_vacation_details(self):
        return f'Остаток отпускных дней: {self.remaining_vacation_days}.'
    
    def __generate_employee_id(self):
        return hash(self.first_name + self.second_name + self.gender)


class FullTimeEmployee(Employee):
    def __init__(self, first_name, second_name, gender, salary):
        super().__init__(first_name, second_name, gender)
        self.__salary = salary

    def get_unpaid_vacation(self, start_date, days):
        return f'Начало неоплачиваемого отпуска: {start_date}, продолжительность: {days} дней.'
    
    def __get_vacation_salary(self):
        return 0.8 * self.__salary

class PartTimeEmployee(Employee):
    vacation_days = 14

    def __init__(self, first_name, second_name, gender):
        super().__init__(first_name, second_name, gender)
        self.remaining_vacation_days = PartTimeEmployee.vacation_days


# Пример использования:
full_time_employee = FullTimeEmployee('Иван', 'Иванов', 'м', 50000)
print(full_time_employee.get_unpaid_vacation('2023-07-01', 5))

part_time_employee = PartTimeEmployee('Анна', 'Петрова', 'ж')
part_time_employee.consume_vacation(5)
print(part_time_employee.get_vacation_details())


# Задание 2

**Описание:** Создайте иерархию классов для различных типов транспортных средств (Необходим один родительский класс и 3 дочерних). Реализуйте метод, который позволяет каждому транспортному средству возвращать собственное описание (Метод в каждом классе должен иметь одинаковое название). Продемонстрируйте вызов данного метода для каждого транспортного средства.


Отрабатываемый принцип: Полиморфизм

In [ ]:
class Transport():
    def __init__(self, name):
        self.name = name


class Car(Transport):
    def get_info(self):
        return f'Машина {self.name}, может ехать'


class Plane(Transport):
    def get_info(self):
        return f'Самолёт {self.name}, может летать'


class Ship(Transport):
    def get_info(self):
        return f'Судно {self.name}, может плыть (и тонуть)'


In [ ]:
print(Car('Lada').get_info())
print(Plane('Airbus').get_info())
print(Ship('Titanic').get_info())

# Задание 3

Онлайн-магазин:
- Создайте модель для онлайн-магазина с классами Product, Order, Customer, и ShoppingCart.
- Product включает информацию о цене, наличии на складе и категории товара.
Order обрабатывает процесс покупки, включая расчет цены с учетом скидок и налогов.
- Customer управляет информацией о пользователе и его истории заказов.
- ShoppingCart позволяет добавлять, удалять и обновлять количество товаров перед оформлением заказа.


In [ ]:
class Product():
    def __init__(self, name: str, price: float, category: str, amount: int):
        self.name = name
        self.price = price
        self.category = category
        self.amount = amount

    def is_available(self):
        return self.amount > 0


class Catalog():
    def __init__(self, products: list[Product]):
        self.products = products
        self.products_names = self.get_products_names(self.products)
        self.products_categories = self.get_products_categories(self.products)
        self.products_prices = self.get_products_prices(self.products)
        self.products_amounts = self.get_products_amounts(self.products)

    def get_products_names(self, products):
        names: list[str] = []
        for product in products:
            names.append(product.name)
        return names

    def get_products_categories(self, products):
        categories: list[str] = []
        for product in products:
            category = product.category
            if category not in categories:
                categories.append(product.category)
        return categories

    def get_products_prices(self, products):
        prices: list[float] = []
        for product in products:
            prices.append(product.price)
        return prices

    def get_products_amounts(self, products):
        amounts: list[int] = []
        for product in self.products:
            amounts.append(product.amount)
        return amounts

    def find_product(self, product_name: str):
        product_index = self.products_names.index(product_name)
        return self.products[product_index]

    def filter_products_names_by_category(self, category):
        return self.get_products_names(list(filter(
            lambda product: product.category == category,
            self.products
        )))


class Customer():
    def __init__(self, name: str):
        self.name = name.capitalize()
        self.order_history = []

    def get_info(self):
        return f'Имя: {self.name}\nИстория заказов: {self.order_history}'

    def add_order(self, order):
        self.order_history.append(
            Catalog(list(order.shopping_cart.cart.keys())).products_names)


class PromoCode():
    def __init__(self, name: str, discount: float):
        self.name = name
        self.discount = discount
        self.is_used = False

    def check(self):
        if self.is_used:
            return None
        elif not self.is_used:
            return self

    def use(self):
        self.is_used = True


class ShoppingCart():
    def __init__(self):
        self.cart: dict[Product, int] = {}

    def add_product(self, product: Product):
        if not product.is_available():
            return f'Товара "{product.name}" нет в наличии'
        elif product.is_available():
            if product in self.cart:
                self.cart[product] += 1
            elif product not in self.cart:
                self.cart[product] = 1
            product.amount -= 1
            return f'"{product.name}" добавлено в корзину'

    def remove_product(self, product: Product):
        if product not in self.cart:
            return f'У вас нет товара "{product.name}" в корзине'
        elif product in self.cart:
            self.cart[product] -= 1
            if self.cart[product] == 0:
                del self.cart[product]
            product.amount += 1
            return f'"{product.name}" удалено из корзины'

    def update_product_amount(self, product: Product, amount: int):
        try:
            product_amount_in_cart = self.cart[product]
        except KeyError:
            text = f'У вас нет товара "{product.name}" в корзине.'
        else:
            delta_amount = abs(product_amount_in_cart - amount)
            if amount > product_amount_in_cart:
                for _ in range(delta_amount):
                    text = self.add_product(product)
                    if not product.is_available():
                        if _ == 0:
                            text += f' {delta_amount} шт'
                        elif _ > 0:
                            text += f' {_ + 1} шт'
                        break
                else:
                    text += f' {_ + 1} шт'
            elif amount < product_amount_in_cart:
                for _ in range(delta_amount):
                    text = self.remove_product(product)
                    if product not in self.cart:
                        if _ == 0:
                            text += f' {delta_amount} шт'
                        elif _ > 0:
                            text += f' {_ + 1} шт'
                        break
                else:
                    text += f' {_ + 1} шт'
            elif delta_amount == 0:
                return ''
        return text

    def clear_cart(self):
        self.cart.clear()

    def print_cart(self):
        if not self.cart:
            print('В корзине пока что пусто')
        else:
            longest_product_name_len = len(
                max(self.cart.keys(),
                    key=lambda product: len(product.name)).name)
            indent_size = max(12, longest_product_name_len)
            print(
                'Наименование'.ljust(indent_size),
                'Количество',
                'Стоимость (без учёта скидок и налога)'
            )
            for product, amount in self.cart.items():
                print(
                    product.name.ljust(indent_size),
                    str(amount).ljust(10),
                    product.price * amount
                )


class Order():
    def __init__(self,
                 shopping_cart: ShoppingCart,
                 customer: Customer,
                 promo: PromoCode | None = None):
        self.tax = 0.2
        self.shopping_cart = shopping_cart
        self.promo = promo.check() if promo else None
        self.total_price = self.calculate_total_price()
        self.customer = customer


    def calculate_total_price(self):
        cart = self.shopping_cart.cart
        subtotal = sum(
            product.price * amount for product, amount in cart.items()
        )
        if self.promo is None:
            discount = 0
        else:
            discount = self.promo.discount / 100
        discount_amount = subtotal * discount
        tax_amount = (subtotal - discount_amount) * self.tax
        return subtotal - discount_amount + tax_amount

    def place(self):
        self.customer.add_order(self)
        self.shopping_cart.clear_cart()
        if self.promo:
            self.promo.use()
        print('Заказ оформлен')


In [ ]:
products_data = [
  {"name": "яблоко", "price": 10, "category": "фрукты", "amount": 10},
  {"name": "банан", "price": 12, "category": "фрукты", "amount": 11},
  {"name": "апельсин", "price": 15, "category": "фрукты", "amount": 12},
  {"name": "груша", "price": 14, "category": "фрукты", "amount": 13},
  {"name": "виноград", "price": 18, "category": "фрукты", "amount": 14},
  {"name": "персик", "price": 20, "category": "фрукты", "amount": 15},
  {"name": "ананас", "price": 25, "category": "фрукты", "amount": 16},
  {"name": "киви", "price": 22, "category": "фрукты", "amount": 17},
  {"name": "лимон", "price": 11, "category": "фрукты", "amount": 18},
  {"name": "манго", "price": 30, "category": "фрукты", "amount": 19},
  {"name": "картофель", "price": 8, "category": "овощи", "amount": 20},
  {"name": "морковь", "price": 7, "category": "овощи", "amount": 21},
  {"name": "помидор", "price": 10, "category": "овощи", "amount": 22},
  {"name": "огурец", "price": 9, "category": "овощи", "amount": 23},
  {"name": "лук", "price": 6, "category": "овощи", "amount": 24},
  {"name": "капуста", "price": 5, "category": "овощи", "amount": 25},
  {"name": "чеснок", "price": 12, "category": "овощи", "amount": 26},
  {"name": "перец", "price": 13, "category": "овощи", "amount": 27},
  {"name": "баклажан", "price": 14, "category": "овощи", "amount": 28},
  {"name": "шпинат", "price": 10, "category": "овощи", "amount": 29},
  {"name": "курица", "price": 50, "category": "мясо", "amount": 30},
  {"name": "говядина", "price": 70, "category": "мясо", "amount": 10},
  {"name": "свинина", "price": 60, "category": "мясо", "amount": 11},
  {"name": "баранина", "price": 80, "category": "мясо", "amount": 12},
  {"name": "индейка", "price": 55, "category": "мясо", "amount": 13},
  {"name": "рыба", "price": 40, "category": "мясо", "amount": 14},
  {"name": "молоко", "price": 20, "category": "молочные продукты", "amount": 15},
  {"name": "сыр", "price": 30, "category": "молочные продукты", "amount": 16},
  {"name": "йогурт", "price": 15, "category": "молочные продукты", "amount": 17},
  {"name": "масло", "price": 25, "category": "молочные продукты", "amount": 18}
]

promo_codes_data = [
    {"name": "PROMO2024_1", "discount": 15},
    {"name": "PROMO2024_2", "discount": 25},
    {"name": "PROMO2024_3", "discount": 30},
    {"name": "PROMO2024_4", "discount": 45},
    {"name": "PROMO2024_5", "discount": 10}
]

catalog = Catalog([Product(**product_data)
                   for product_data in products_data])

promos = [PromoCode(**promo_data) for promo_data in promo_codes_data]

shopping_cart = ShoppingCart()

COMMANDS = [
    "Помощь",
    "Выйти",
    "Корзина",
    "Каталог",
    "Профиль",
    "оформить заказ"
]

In [ ]:
def registration(name: str):
    global customer
    customer = Customer(name)


def create_order(shopping_cart, customer, promo):
    global current_order
    current_order = Order(shopping_cart, customer, promo)


def command_help():
    return f'Список доступных команд {COMMANDS}'


def handler(data):

    message = data['text']

    if message is None:
        text = (
            'Добро пожаловать в онлайн-магазин\n'
            'Чтобы зарегистрироваться используйте команду "Регистрация"'
        )

    elif not message:
        text = ''

    elif message == 'выйти':
        data['end_of_session'] = True
        text = 'До скорых встреч!'

    elif message == 'регистрация':
        data['context'] = 'registration'
        text = 'Введите ваше имя:'

    elif data['context'] == 'registration':
        name = message
        registration(name)
        text = f'Регистрация прошла успешно\n{command_help()}'
        data['context'] = 'main'

    elif customer is None and data['context'] != 'registration':
        text = 'Для начала зарегистрируйтесь (используйте команду Регистрация)'

    elif message == 'помощь':
        text = command_help()

    elif message == 'корзина':
        shopping_cart.print_cart()
        if shopping_cart.cart:
            data['context'] = 'shopping_cart'
            text = 'Чтобы изменить количество продукта используйте команду "Изменить"'
        else:
            text = ''

    elif message == 'каталог':
        data['context'] = 'catalog'
        text = f'Выберете категорию\n{catalog.products_categories}'

    elif message == 'профиль':
        text = customer.get_info()

    elif message == 'оформить заказ':
        data['context'] = 'order'
        text = ('У вас есть доступные промокоды\n'
                'Чтобы узнать подробнее используйте команду "Промокоды"\n'
                'Чтобы продолжить без промокода напишите "Продолжить"')

    else:
        text = 'Не удалось распознать команду'

        if data['context'] == 'shopping_cart':
            if message == 'изменить':
                text = 'Введите название продукта и необходимое количество через запятую'
            elif (
                len(message.split(', ')) == 2
                and (product_name := message.split(', ')[0]) in catalog.products_names
                and (amount := int(message.split(', ')[1])) >= 0
            ):
                text = shopping_cart.update_product_amount(
                    catalog.find_product(product_name), amount
                )

        elif data['context'] == 'catalog':
            if message in catalog.products_categories:
                category = message
                text = (
                    'Выберете продукт\n'
                    f'{catalog.filter_products_names_by_category(category)}'
                )
            elif message in catalog.products_names:
                product_name = message
                product = catalog.find_product(product_name)
                text = shopping_cart.add_product(product)

        elif data['context'] == 'order':
            global current_order
            if message == 'промокоды':
                text = ''
                for promo in promos:
                    text = '\n'.join(
                        (text, f'{promo.name} - {promo.discount}%')
                    )
            elif (
                (promo_name := message.upper()) in (
                    promos_names := [promo.name for promo in promos]
                )
                or message == 'продолжить'
            ):
                if message == 'продолжить':
                    promo = None
                else:
                    promo_index = promos_names.index(promo_name)
                    promo = promos[promo_index]

                create_order(shopping_cart, customer, promo)
                text = (
                    f'Заказ на сумму {current_order.total_price} '
                    'готов к оформлению\n'
                    'Чтобы завершить оформление используйте команду "Завершить"'
                )

            elif message == 'завершить':
                current_order.place()
                current_order = None
                text = ''

    return text


def main():
    data = {
        'context': 'main',
        'end_of_session': False,
        'text': None
    }
    while True:
        print(handler(data))
        if data['end_of_session']:
            break
        message = input('> ').lower()
        data['text'] = message


customer = None
current_order = None

if __name__ == '__main__':
    main()


# Задание 4

Симулятор космического корабля:
- Создайте симулятор управления космическим кораблем с классами SpaceShip, CrewMember, и Mission.
- SpaceShip имеет атрибуты для управления топливом, состоянием корпуса, и текущей скоростью.
- CrewMember контролирует здоровье, навыки, и роли в команде (например, пилот, инженер).
- Mission определяет цели, ресурсы, и возможные события (например, аварии, встречи с астероидами).

In [ ]:
class SpaceShip():
    def __init__(self, crew):
        self.crew = crew
        self.crew.ship = self
        self.fuel = 100
        self.fuel_consumption = 0
        self.hull = 100
        self.current_velocity = 0
        self.max_velovity = 100
        self.is_destroyed = False

    def change_velocity(self, delta_velocity):
        if self.ship.is_destroyed:
            print('Корабль уничтожен.')
        elif abs(self.current_velocity + delta_velocity) > self.max_velovity:
            print('Достигнут максимум скорости.')
        else:
            self.current_velocity += delta_velocity
            self.fuel_consumption += delta_velocity / 10

    def move(self, distance):
        if self.fuel <= 0:
            print('Топливо закончилось.')
        elif self.current_velocity == 0:
            print('Чтобы начать движение добавьте скорость.')
        elif self.fuel_consumption * distance >= self.fuel:
            print('У вас не хватит топлива.')
        else:
            self.fuel -= self.fuel_consumption * distance

    def hit(self, damage):
        if damage >= self.hull:
            self.destroy()
        else:
            self.hull -= damage

    def destroy(self):
        self.is_destroyed = True


class CrewMember():
    def __init__(self, role, skills):
        self.ship = None
        self.health = 100
        self.is_alive = True
        self.skills_level = 0
        self.skills = skills
        self.role = role

    def use_skill(self, skill):
        if self.ship.is_destroyed:
            print('Корабль уничтожен.')
        elif skill not in self.skills:
            print('Этот член экипажа не обладает этим навыком.')
        elif skill['name'] == 'reparing':
            self.ship.hull += 5 * (1 + self.skills_level / 100)

    def hit(self, damage):
        if damage >= self.health:
            self.die()
        else:
            self.health -= 10

    def die(self):
        self.is_alive = False


class Mission():
    def __init__(self, target, resources, event):
        self.target = target
        self.resources = resources
        self.event = event


missions = []


In [ ]:
import random


def main():
    pass


if __name__ == '__main__':
    main()

# Дополнительно:

**Описание:** создайте консольную версию игры крестики-нолики, используя классы